In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import re
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from statistics import mean
from sklearn.decomposition import TruncatedSVD
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [2]:
link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/finaldataset.csv'
data = pd.read_csv(link)
data=data.drop(['title_x','title_y','title','value'],axis=1)
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,কিছুটা নেতিবাচক
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,কিছুটা নেতিবাচক
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,নিশ্চিত নেতিবাচক
3,চুরি নয় লুটপাট।,নিশ্চিত নেতিবাচক
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,নিরপেক্ষ


In [3]:
le = LabelEncoder()
data['tag'] = le.fit_transform(data['tag'])
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,1
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,1
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,4
3,চুরি নয় লুটপাট।,4
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2


In [4]:
data['tag']=data['tag'].replace(3, 0) #Positive Class
data['tag']=data['tag'].replace(4, 1) #Negative Class
data = data[data['tag'] != 2]#Dropping the neutral class

In [5]:
d = data['data'].tolist()
label = data['tag'].tolist()

In [6]:
for i in range(len(d)):
    d[i] = re.sub(r'[^\w\s]', '', d[i])

In [7]:
vectorizer = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(d).toarray()

In [8]:
tsvd = TruncatedSVD(n_components=X.shape[1]-1)
X_tsvd = tsvd.fit(X)
tsvd_var_ratios = tsvd.explained_variance_ratio_
def select_n_components(var_ratio, goal_var: float) -> int:
# Set initial variance explained so far
    total_variance = 0.0
# Set initial number of features
    n_components = 0
# For the explained variance of each feature:
    for explained_variance in var_ratio:
# Add the explained variance to the total
        total_variance += explained_variance
# Add one to the number of components
        n_components += 1
# If we reach our goal level of explained variance
        if total_variance >= goal_var:
# End the loop
            break
# Return the number of components
    return n_components

In [9]:
i = select_n_components(tsvd_var_ratios, 0.95)

In [10]:
svd = TruncatedSVD ( n_components= i ) 
# prepare transform on dataset
svd.fit(X)
# apply transform to dataset
X = svd.transform(X)
a = np.asarray(X)
np.savetxt("BOWBOW_1_1_svd.csv", a, delimiter=",")

In [11]:
print(i)

3849


# TF-IDF

In [10]:
vectorizer_1 = TfidfVectorizer(ngram_range=(1, 1))
X_1 = vectorizer_1.fit_transform(d).toarray()
print(X_1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
vectorizer_2 = TfidfVectorizer(ngram_range=(1,2))
X_2 = vectorizer_2.fit_transform(d).toarray()
print(X_2)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
